In [1]:
!pip install git+https://github.com/ContinualAI/avalanche.git

  Cloning https://github.com/ContinualAI/avalanche.git to /tmp/pip-req-build-ytu4k_iq
  Running command git clone -q https://github.com/ContinualAI/avalanche.git /tmp/pip-req-build-ytu4k_iq
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 532 kB 33.8 MB/s 
     |████████████████████████████████| 121 kB 71.3 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 1.7 MB 35.1 MB/s 
     |████████████████████████████████| 139 kB 56.2 MB/s 
     |████████████████████████████████| 180 kB 49.7 MB/s 
     |████████████████████████████████| 97 kB 7.1 MB/s 
     |████████████████████████████████| 63 kB 1.9 MB/s 
  Created wheel for avalanche-lib: filename=avalanche_lib-0.0.1-py3-none-any.whl size=414217 sha256=1eb709591ab10aca7080c53ef64ca4b444654cd075c34d595dd3ac2c

In [13]:
import torch
from torch.nn import CrossEntropyLoss
from torch.optim import SGD
import torchvision
from torchvision import transforms

from avalanche.benchmarks.classic import SplitOmniglot
from avalanche.models import SimpleMLP
from avalanche.training.strategies import Naive
from avalanche.benchmarks.utils import ImageFolder, DatasetFolder, FilelistDataset, AvalancheDataset
from avalanche.benchmarks.scenarios.new_classes.nc_scenario import NCScenario

# Using Avalanche's built-in Omniglot dataset

In [31]:
height = 32
n_classes = 964
batch_size = 8
epochs = 5

benchmark = SplitOmniglot(n_experiences=4, 
                          seed=1, 
                          train_transform=transforms.Compose([
                              transforms.Resize(height),
                              transforms.Grayscale(),
                              transforms.ToTensor(),
                              transforms.Normalize((0.9221,), (0.2681,))
                          ]), 
                          eval_transform=transforms.Compose([
                              transforms.Resize(height),
                              transforms.Grayscale(),
                              transforms.ToTensor(),
                              transforms.Normalize((0.9221,), (0.2681,))
                          ]))

model = SimpleMLP(num_classes=n_classes, input_size=height*height)

optimizer = SGD(model.parameters(), lr=0.001, momentum=0.9)
criterion = CrossEntropyLoss()

cl_strategy = Naive(
    model, optimizer, criterion, train_mb_size=batch_size, train_epochs=epochs,
    eval_mb_size=batch_size, device=device)

print('Starting experiment...')
results = []
for experience in benchmark.train_stream:
    print("Start of experience: ", experience.current_experience)
    print("Current Classes: ", experience.classes_in_this_experience)

    cl_strategy.train(experience)
    print('Training completed')

    print('Computing accuracy on the whole test set')
    results.append(cl_strategy.eval(benchmark.test_stream))

Files already downloaded and verified
Files already downloaded and verified
Starting experiment...
Start of experience:  0
Current Classes:  [2, 8, 520, 10, 529, 531, 534, 28, 541, 542, 33, 35, 548, 550, 40, 554, 556, 47, 559, 561, 564, 567, 568, 58, 571, 573, 62, 574, 64, 580, 70, 72, 74, 589, 78, 590, 592, 593, 594, 595, 598, 87, 600, 90, 93, 607, 97, 100, 102, 614, 617, 107, 623, 626, 115, 628, 122, 123, 636, 126, 638, 129, 647, 139, 651, 653, 143, 144, 145, 146, 655, 148, 656, 657, 659, 152, 664, 668, 157, 161, 163, 676, 166, 679, 168, 681, 171, 685, 174, 686, 693, 696, 698, 187, 699, 191, 705, 706, 195, 196, 707, 708, 199, 200, 710, 202, 716, 721, 211, 212, 213, 728, 729, 218, 733, 223, 738, 739, 228, 229, 233, 241, 756, 761, 762, 763, 252, 253, 764, 255, 766, 770, 774, 775, 264, 778, 267, 269, 271, 784, 786, 789, 791, 283, 284, 796, 287, 289, 801, 804, 805, 295, 809, 812, 813, 308, 309, 310, 821, 312, 827, 829, 318, 833, 322, 835, 325, 837, 327, 840, 841, 330, 331, 847, 337, 849,

# Experiment with loading own dataset

In [3]:
dataset = torchvision.datasets.Omniglot(
    root="./data", download=True, transform=torchvision.transforms.ToTensor()
)

image, label = dataset[0]
print(type(image))  # torch.Tensor
print(type(label))  # int

  0%|          | 0/9464212 [00:00<?, ?it/s]

Extracting ./data/omniglot-py/images_background.zip to ./data/omniglot-py
<class 'torch.Tensor'>
<class 'int'>


In [9]:
height = 32
TRANSFORM_IMG = transforms.Compose([
  transforms.Resize(height),
  transforms.Grayscale(),
  transforms.ToTensor()
])

ds = ImageFolder('./data/omniglot-py/images_background/Korean', transform=TRANSFORM_IMG)
print(ds)

Dataset ImageFolder
    Number of datapoints: 800
    Root location: ./data/omniglot-py/images_background/Korean
    StandardTransform
Transform: Compose(
               Resize(size=32, interpolation=bilinear, max_size=None, antialias=None)
               Grayscale(num_output_channels=1)
               ToTensor()
           )


In [11]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
n_classes = 40
batch_size = 8
epochs = 5
n_experiences = 20

model = SimpleMLP(num_classes=n_classes, input_size=height*height)

scenario = NCScenario(ds, ds, n_experiences=n_experiences, task_labels=n_classes)
train_stream = scenario.train_stream
test_stream = scenario.test_stream

optimizer = SGD(model.parameters(), lr=0.001, momentum=0.9)
criterion = CrossEntropyLoss()

cl_strategy = Naive(
    model, optimizer, criterion, train_mb_size=batch_size, train_epochs=epochs,
    eval_mb_size=batch_size, device=device)

results = []
for train_task in train_stream:
    print("Start of experience: ", train_task.current_experience)
    print("Current Classes: ", train_task.classes_in_this_experience)
    print("Classes seen so far: ", train_task.classes_seen_so_far)
    print()

    cl_strategy.train(train_task)
    print('Training completed')

    print('Computing accuracy on the whole test set')
    results.append(cl_strategy.eval(test_stream))


Start of experience:  0
Current Classes:  [1, 36]
Classes seen so far:  [1, 36]

-- >> Start of training phase << --
-- Starting training on experience 0 (Task 0) from train stream --
100%|██████████| 10/10 [00:00<00:00, 97.89it/s] 
Epoch 0 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 2.6110
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.3250
100%|██████████| 10/10 [00:00<00:00, 104.67it/s]
Epoch 1 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 0.8129
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.5750
-- >> End of training phase << --
Training completed
Computing accuracy on the whole test set
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --
100%|██████████| 10/10 [00:00<00:00, 126.86it/s]
> Eval on experience 0 (Task 0) from test stream ended.
	Loss_Exp/eval_phase/test_stream/Task000/Exp000 = 0.7669
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp000 = 0.5000
-- Starting eval on experience 1 (Task 1) from test s